# Importing Packages

In [21]:
import requests
import json
from io import StringIO
from bs4 import BeautifulSoup as bs
from datetime import date, timedelta
import covidcast

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 100)

import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

import pickle

from sklearn.pipeline import Pipeline 
from sklearn.impute import KNNImputer, SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, accuracy_score, f1_score, confusion_matrix
from xgboost import XGBClassifier


# Data Collection and Exploratory Data Analysis


## NYT Mask Wearing Survey Data

- This data comes from a large number of interviews conducted online by the firm Dynata, which asked the question to obtain 250,000 survey responses between July 2 and July 14, enough data to provide estimates more detailed than the state level. 
- This survey was conducted for a single run and there are no plans for a follow-up study.
- Specifically, each participant was asked the following question: 

<center><strong>How often do you wear a mask in public when you expect to be within six feet of another person?</strong></center>

- Here are the definitions for the column headings:

    - **COUNTYFP**: The county FIPS code.
    - **NEVER**: The estimated share of people in this county who would say never in response to the question: “How often do you wear a mask in public when you expect to be within six feet of another person?”
    - **RARELY**: The estimated share of people in this county who would say rarely
    - **SOMETIMES**: The estimated share of people in this county who would say sometimes
    - **FREQUENTLY**: The estimated share of people in this county who would say frequently
    - **ALWAYS**: The estimated share of people in this county who would say always

 - In their analysis, they assumed the following to be true:  ALWAYS --> 100%, FREQUENTLY --> 80%, SOMETIMES --> 50%, RARELY --> 20%, NEVER --> 0%




In [23]:
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv'
s = requests.get(url).text
nymask = pd.read_csv(StringIO(s))
nymask.COUNTYFP = nymask.COUNTYFP.astype(str)
nymask.COUNTYFP = np.where(nymask['COUNTYFP'].str.len() == 4, '0' + nymask.COUNTYFP, nymask.COUNTYFP)
nymask.columns = ['FIPS', 'NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS']
nymask.head()

,FIPS,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,01001,0.053,0.074,0.134,0.295,0.444
1,01003,0.083,0.059,0.098,0.323,0.436
2,01005,0.067,0.121,0.120,0.201,0.491
3,01007,0.020,0.034,0.096,0.278,0.572
4,01009,0.053,0.114,0.180,0.194,0.459


In [106]:
nymask.head()

,FIPS,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,01001,0.053,0.074,0.134,0.295,0.444
1,01003,0.083,0.059,0.098,0.323,0.436
2,01005,0.067,0.121,0.120,0.201,0.491
3,01007,0.020,0.034,0.096,0.278,0.572
4,01009,0.053,0.114,0.180,0.194,0.459


In [111]:
nymask.loc[nymask['FIPS'] == '10003']

,FIPS,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
317,10003,0.027,0.013,0.041,0.133,0.786


<img src='images/fb_mask_data.png'>

## Carnegie Mellon Mask Wearing Facebook Survey Data

Between the new survey’s deployment on September 8, 2020, and October 7th, the study collected 1,220,000 valid responses from respondents across the United States. 
Each response includes questions about symptoms, mask wearing, testing, and the other important topics described above, along with demographic details about the respondent. These demographics include age, gender, race, occupation, and education, allowing us to understand how different groups have been affected and which groups are currently most vulnerable to COVID-19. 

- **smoothed_wearing_mask**
  - Estimated percentage of people who wore a mask for most or all of the time while in public in the past 5 days; those not in public in the past 5 days are not counted.	

- **smoothed_others_masked**
  - Estimated percentage of respondents who say that most or all other people wear masks, when they are in public and social distancing is not possible


In [24]:
# create variable to hold date object for two days ago
two_days_ago = date.today() - timedelta(days=2)

In [25]:
# get the most recent survey data
mask_ind = covidcast.signal("fb-survey", "smoothed_wearing_mask",
                            date(2020, 10, 1), two_days_ago,
                            "county")
mask_oth = covidcast.signal('fb-survey', 'smoothed_others_masked', 
                            date(2020, 11, 24),two_days_ago, 
                            "county")

In [26]:
# remove data that isn't at the county level
mask_fip = mask_ind.loc[~mask_ind.geo_value.str.endswith('000')]
mask_oth = mask_oth.loc[~mask_oth.geo_value.str.endswith('000')]

In [27]:
# check number of counties represented 
print(mask_ind.geo_value.value_counts().shape)
print(mask_fip.geo_value.value_counts().shape)
mask_oth.geo_value.value_counts().shape

(726,)
(677,)


(655,)

In [28]:
# pare down datasets to just columns of interest
mask_ind_means = pd.DataFrame(mask_fip.groupby(['geo_value'])['value'].mean()).reset_index()
mask_ind_means.columns = ['FIPS', 'ind_mask']
mask_ind_means.FIPS = mask_ind_means.FIPS.apply(lambda x: x.zfill(5))
# repeat for other survey
mask_oth_means = pd.DataFrame(mask_oth.groupby(['geo_value'])['value'].mean()).reset_index()
mask_oth_means.columns = ['FIPS', 'oth_mask']
mask_oth_means.FIPS = mask_oth_means.FIPS.apply(lambda x: x.zfill(5))

<img src='images/delphi_dec.png'>

In [29]:
mask_means = mask_ind_means.merge(mask_oth_means, on = 'FIPS', how = 'outer')
mask_w_nyt = mask_means.merge(nymask, on = 'FIPS', how = 'outer')

In [ ]:
mask_means

In [34]:
mask_w_nyt

,FIPS,ind_mask,oth_mask,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,01003,79.671626,52.867505,0.083,0.059,0.098,0.323,0.436
1,01051,88.291970,65.708307,0.042,0.095,0.127,0.252,0.485
2,01069,85.591414,57.060285,0.085,0.079,0.135,0.268,0.433
3,01073,91.454122,79.596565,0.049,0.037,0.107,0.179,0.628
4,01081,89.816314,71.553968,0.053,0.064,0.138,0.183,0.562
...,...,...,...,...,...,...,...,...
3137,56037,NaN,NaN,0.061,0.295,0.230,0.146,0.268
3138,56039,NaN,NaN,0.095,0.157,0.160,0.247,0.340
3139,56041,NaN,NaN,0.098,0.278,0.154,0.207,0.264
3140,56043,NaN,NaN,0.204,0.155,0.069,0.285,0.287


## Johns Hopkins COVID Data Geopandas

This feature layer contains the most up-to-date COVID-19 cases for the US. Data is pulled from the Coronavirus COVID-19 Global Cases by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University, the Red Cross, the Census American Community Survey, and the Bureau of Labor and Statistics, and aggregated at the US county level. 

This web map created and maintained by the Centers for Civic Impact at the Johns Hopkins University, and is supported by the Esri Living Atlas team and JHU Data Services. It is used in the COVID-19 United States Cases by County dashboard. 

 For more information on Johns Hopkins University’s response to COVID-19, visit the Johns Hopkins Coronavirus Resource Center where our experts help to advance understanding of the virus, inform the public, and brief policymakers in order to guide a response, improve care, and save lives.


In [30]:
geo = gpd.read_file('https://opendata.arcgis.com/datasets/4cb598ae041348fb92270f102a6783cb_0.geojson')
print(geo.shape)
geo.head()

(3331, 88)


,OBJECTID,Countyname,ST_Name,ST_Abbr,ST_ID,FIPS,FatalityRa,Confirmedb,DeathsbyPo,PCTPOVALL_,Unemployme,Med_HH_Inc,State_Fata,DateChecke,EM_type,EM_date,EM_notes,url,Thumbnail,Confirmed,Deaths,Age_85,Age_80_84,Age_75_79,Age_70_74,Age_65_69,Beds_Licen,Beds_Staff,Beds_ICU,Ventilator,POP_ESTIMA,POVALL_201,Unemployed,Median_Hou,Recovered,Active,State_Conf,State_Deat,State_Reco,State_Test,AgedPop,NewCases,NewDeaths,TotalPop,NonHispWhP,BlackPop,AmIndop,AsianPop,PacIslPop,OtherPop,TwoMorPop,HispPop,Wh_Alone,Bk_Alone,AI_Alone,As_Alone,NH_Alone,SO_Alone,Two_More,Not_Hisp,Age_Less15,Age_15_24,Age_25_34,Age_Over75,Agetotal,NonHisp,Age_35_64,Age_65_74,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,NewCasebyP,Inpat_Occ,ICU_Occ,Shape__Area,Shape__Length,geometry
0,1,Autauga,Alabama,AL,01,01001,1.076426,8354.17,89.926440,13.8,3.6,118.9591,1.299898,01/07/2021 11:30:49,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,4645,50,815,1026,1498,2440,2271,85,55,6,2,55601,7587,942,59338,0,0,384184,4994,0,1997125,8050,99,0,55200,41412,10475,159,568,5,41,1012,1528,42437,10565,159,568,32,409,1030,53672,10842,7192,7064,3339,55200,1528,22052,4711,99.0,210.0,31.0,37.0,29.0,49.0,26.0,59.0,40.0,36.0,30.0,9.0,48.0,53.0,178.054352,96.320346,100.000000,2.209382e+09,246839.865479,"POLYGON ((-86.41312 32.70739, -86.41219 32.526..."
1,2,Baldwin,Alabama,AL,01,01003,1.166758,6722.26,78.432452,9.8,3.6,115.4508,1.299898,01/07/2021 11:30:49,Govt Ordered Community Quarantine,4/3/2020 10:41:19 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,14656,171,3949,4792,7373,11410,13141,386,362,51,8,218022,21069,3393,57588,0,0,384184,4994,0,1997125,40665,216,2,208107,172768,19529,1398,1668,9,410,2972,9353,179526,19764,1522,1680,9,2034,3572,198754,37621,23497,23326,16114,208107,9353,82998,24551,216.0,253.0,123.0,109.0,132.0,222.0,209.0,220.0,210.0,137.0,117.0,42.0,145.0,200.0,99.072571,74.029401,120.089286,5.770469e+09,728445.072448,"MULTIPOLYGON (((-87.78878 31.29877, -87.78849 ..."
2,3,Barbour,Alabama,AL,01,01005,2.191609,6418.55,140.669587,30.9,5.2,68.9280,1.299898,01/07/2021 11:30:49,Govt Ordered Community Quarantine,4/3/2020 10:43:21 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,1597,35,422,551,841,1305,1515,74,30,5,2,24881,6788,433,34382,0,0,384184,4994,0,1997125,4634,22,2,25782,11898,12199,63,85,1,86,344,1106,12216,12266,72,96,1,778,353,24676,4517,3092,3675,1814,25782,1106,9864,2820,22.0,42.0,3.0,2.0,11.0,3.0,22.0,30.0,45.0,11.0,8.0,2.0,6.0,7.0,88.420883,56.711409,88.571429,3.258643e+09,307285.154510,"POLYGON ((-85.25609 32.13767, -85.25569 32.135..."
3,4,Bibb,Alabama,AL,01,01007,2.469136,8678.57,214.285714,21.8,4.0,92.3478,1.299898,01/07/2021 11:30:49,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,1944,48,427,488,624,842,1280,35,25,4,1,22400,4400,344,46064,0,0,384184,4994,0,1997125,3661,21,2,22527,16801,4974,8,37,0,0,160,547,17268,5018,8,37,0,9,187,21980,3742,3005,3075,1539,22527,547,9044,2122,21.0,38.0,3.0,19.0,9.0,20.0,17.0,25.0,30.0,16.0,7.0,14.0,14.0,28.0,93.750000,19.047619,NaN,2.310715e+09,227886.963840,"POLYGON ((-87.02685 33.24646, -87.02572 33.209..."
4,5,Blount,Alabama,AL,01,01009,1.367905,8468.19,115.836791,13.2,3.5,101.0645,1.299898,01/07/2021 11:30:49,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,4898,67,866,1459,1776,2802,3330,25,25,6,2,57840,7527,878,50412,0,0,384184,4994,

- FatalityRa = County Fatality Rate
- Confirmedb = County Confirmed divided by County Population * 100,000
- DeathsbyPo = County Deaths divided by County Population * 100,000
- Unemployme = Unemployment Rate
- EM_type, EM_date, EM_notes - could be turned into a column about Emergency Declaration
- Confirmed = County Level Confirmed Cases
- Deaths = County Level Deaths
- Beds_Licen = Number of Licensed Beds 
- Beds_Staff = Number of Staffed Beds 
- Beds_ICU = Number of ICU Beds 
- Ventilator = Average Ventilation Used Per Hospital
- POP_ESTIMA = Total Population
- POVALL_201 = Poverty Rate
- Median_Hou - Median Household Income
- Recovered = Number of Recovered Cases
- Active = Number of Active Cases
- AgedPop = TOtal Population Aged 65 Plus
- NewCases = New cases since yesterday
- NewDeaths = Newdeaths since yesterday
- Wh_Alone, Bk_Alone, AI_Alone, As_Alone, NH_Alone, SO_Alone, Two_More, Not_Hisp = Population numbers of each group
- Age_? = Population numbers of age groups
- NewCasebyP = I think this is new case by percentage
- ICU_Occ, Inpat_Occ = Percent Occupied of ICUs and Inpatient Beds
- Shape_Area, Shape_Length = for mapping



In [31]:
# remove all the rows with Unassigned and Out of [State] designations for county name, as well as Puerto Rico
geo = geo[~geo.Countyname.str.contains("Out of")]
geo = geo[~geo.Countyname.str.contains("Unassigned")]
geo = geo[~geo.ST_Name.str.contains("Puerto Rico")]
geo.drop(geo.tail(7).index, inplace=True)

In [32]:
# remove Day_1 to Day_14 columns, unclear what the days represent
geo.drop(columns=['Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6', 'Day_7', 'Day_8', 'Day_9', 'Day_10', 'Day_11', 'Day_12', 'Day_13', 'Day_14'], inplace=True)
# State level data columns were dropped, as well as any ID and repeated population data
geo.drop(columns=['OBJECTID', 'ST_ID', 'Med_HH_Inc', 'State_Fata', 'DateChecke', 'url', 'Thumbnail', 'State_Conf', 'State_Deat', 'State_Reco', 'State_Test'], inplace=True)
# Kept Confirmed and Deaths by county population and removed notes column for Emergency Management
geo.drop(columns=['Confirmed', 'Deaths', 'EM_notes'], inplace=True)
# Removed columns because most values were null
geo.drop(columns=['Recovered', 'Active'], inplace=True)
# Replaced long version of values into shorter abbreviated versions and extracted the date out of the string
geo.EM_type = geo.EM_type.replace({'Govt Ordered Community Quarantine': 'CQ', 'Govt Directed Social Distancing': 'SD'})
geo.EM_date = geo.EM_date.str.extract(r'((\d+)\/(\d{2})\/(\d{4}))')



# Create percentages for races and removed original race columns as well as the other set of racial columns
geo['Wh_Perc'] = geo.NonHispWhP / geo.TotalPop
geo['His_Perc'] = geo.HispPop / geo.TotalPop
geo['Black_Perc'] = geo.BlackPop / geo.TotalPop
geo['API_Perc'] = (geo.AsianPop + geo.PacIslPop) / geo.TotalPop
geo['Native_Perc'] = geo.AmIndop / geo.TotalPop

# Grouped ages into different larger categories
geo['Young_Pop'] = (geo.Age_Less15 + geo.Age_15_24) / geo.Agetotal
geo['Adult_Pop'] = (geo.Age_25_34 + geo.Age_35_64) / geo.Agetotal
geo['Senior_Pop'] = (geo.Age_65_74 + geo.Age_Over75) / geo.Agetotal
# Created some other features
geo['Aged_Perc'] = geo.AgedPop / geo.POP_ESTIMA
geo['Staf_Bed_Perc'] = geo['Beds_Staff'] / geo['Beds_Licen']
geo['Pov_Perc'] = geo.POVALL_201 / geo.POP_ESTIMA
geo['beds_per_capita'] = geo.Beds_ICU / geo.POP_ESTIMA
geo['pop_density'] = geo['TotalPop'] / geo['Shape__Area']


geo.drop(columns=['Age_85', 'Age_80_84', 'Age_75_79', 'Age_70_74', 'Age_65_69', 'Agetotal', 'Age_Less15', 'Age_15_24', 'Age_25_34', 'Age_35_64', 'Age_65_74', 'Age_Over75'], inplace=True)
geo.drop(columns=['Wh_Alone', 'Bk_Alone', 'AI_Alone', 'As_Alone', 'NH_Alone', 'SO_Alone', 'Two_More', 'Not_Hisp', 'NonHispWhP', 'BlackPop', 'AmIndop', 'PacIslPop', 'OtherPop', 'TwoMorPop', 'HispPop', 'NonHisp', 'AsianPop'], inplace=True)
geo.drop(columns=['NewCases', 'NewDeaths', 'AgedPop', 'POVALL_201', 'Unemployed'], inplace=True)
geo.drop(columns=['Beds_Licen', 'Beds_Staff', 'Beds_ICU', 'Ventilator'], inplace=True)

<img src='images/county_conf_by_pop.png'>

<img src='images/county_confirmed.png'>

In [33]:
rural_urban = pd.read_excel('data/ru_code.xls')
fip_rur = rural_urban[['FIPS', 'RUCC_2013']]
fip_rur.FIPS = fip_rur.FIPS.astype(str)
geo = geo.merge(fip_rur, how = 'left', on = 'FIPS')

## 2020 Election Data

In [54]:
df2020 = pd.read_csv('data/2020_Results.csv')
df2020.rename(columns={'county_fips': 'FIPS'}, inplace=True)
df2020.FIPS = df2020.FIPS.astype(str)
df2020.FIPS = df2020.FIPS.apply(lambda x: x.zfill(5))
# # upload shapefile for USA counties
shapefile = 'data/USA_Counties.shp'
gdf = gpd.read_file(shapefile)
gdf = gdf[~gdf.STATE_NAME.isin(['Puerto Rico', 'Alaska', 'Hawaii'])]
gdf.FIPS = gdf.FIPS.astype(str)
merged = gdf.merge(df2020, on='FIPS', how='left')

In [ ]:
# fig, ax = plt.subplots(1, figsize=(20,20))
# merged.plot(column='per_gop', cmap='RdBu_r', linewidth=0.8, ax=ax, edgecolor='black')
# ax.axis('off')
# ax.set_title('Percentage By County for 2020 Election', fontsize=18)
# plt.savefig('images/election2020.png')

<img src='images/election2020.png'>

# Visualizing Combined Data

In [ ]:
merged_2 = merged.merge(nymask, on='FIPS')
merged_2.head()

In [ ]:
merged_2.columns

In [ ]:
df = pd.DataFrame(merged_2)
df.head()

In [ ]:
df.columns

In [ ]:
# Create scatterplots with regression line with regplot() of continuous variables
features = df[['FIPS', 'FatalityRa', 'Confirmedb',
       'DeathsbyPo', 'PCTPOVALL_', 'Unemployme', 'POP_ESTIMA',
       'Median_Hou', 'TotalPop', 'NewCasebyP', 'Inpat_Occ', 'ICU_Occ',
       'Shape__Area', 'Shape__Length', 'Wh_Perc', 'His_Perc',
       'Black_Perc', 'API_Perc', 'Native_Perc', 'Young_Pop', 'Adult_Pop',
       'Senior_Pop', 'Staf_Bed_Perc', 'Aged_Perc', 'Pov_Perc',
       'beds_per_capita', 'pop_density', 'NEVER', 'RARELY',
       'SOMETIMES', 'FREQUENTLY', 'ALWAYS']]
target = df['per_dem']
feat_col = features.columns
feat_col

In [ ]:
df.FIPS = df.FIPS.astype(int)

In [ ]:
features.dtypes

In [ ]:
# con = pd.melt(df, id_vars='per_dem', value_vars=feat_col)
# g = sns.FacetGrid(con, col='variable', col_wrap=3, sharex=False, sharey=False, height=4)
# g = g.map(sns.regplot, 'value', 'per_dem', color='darkorange')
# g.set_xticklabels(rotation=45)
# plt.savefig('images/regplot.png')

<img src='images/regplot.png'>

In [ ]:
df['per_dem_cat'] = df['per_dem']

In [ ]:
df.head()

In [ ]:
condition = [(df['per_dem_cat']) <= .25, 
             (df['per_dem_cat'] > .25) & (df['per_dem_cat'] <= .50),
             (df['per_dem_cat'] > .50) & (df['per_dem_cat'] <= .75),
             (df['per_dem_cat']) > .75]
choices = [0,1,2,3]

df['per_dem_cat'] = np.select(condition, choices, df['per_dem_cat'])

In [ ]:
df.head()

In [80]:
con = pd.melt(df, id_vars='per_dem_cat', value_vars=feat_col)
g = sns.FacetGrid(con, col='variable', col_wrap=3, sharex=False, sharey=False, height=4)
g = g.map(sns.swarmplot, 'value', 'per_dem_cat')
g.set_xticklabels(rotation=45)
# plt.savefig('images/regplot.png')




In [ ]:
## Display distribution plots of continuous variables using FacetGrid and distplot

con_2 = pd.melt(df, value_vars = feat_col)
g = sns.FacetGrid(con_2, col='variable', col_wrap=3, sharex=False, sharey=False, height=4)
g = g.map(sns.distplot, 'value', color='r')
g.set_xticklabels(rotation=45)